# Convergence Plots

## Dependencies

In [ ]:
import json  # noqa: EXE002
import re
import time
from collections import defaultdict
from collections.abc import Iterable
from pathlib import Path
from pprint import pprint

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import pandas as pd

## Parameters

In [ ]:
RESULTS_DIR: Path = Path() / ".." / "data" / "02_results"
OUT_DIR: Path = (
    Path() / ".." / "data" / f"{time.strftime('%Y%m%d%H%M%S')}_convergence"
)
OUT_DIR.mkdir(exist_ok=True, parents=True)

# Set alternative labels to be displayed in the plots' legends
# This is supposed to map from result directory name to the label to be
# displayed.
# Example:
# ALT_LABELS = {
#   rdir.name: f"Test_{i}" for i, rdir in enumerate(RESULTS_DIR.iterdir())
# }
ALT_LABELS: dict[str, str] = {}

## Locating Result Data

In [ ]:
result_paths: dict[str, dict[str, Path]] = {}

res = {
    "bounds": "bounds.csv",
    "dual_solver": "dual_solver.csv",
    "runtime": "runtime.json",
}

for rdir in RESULTS_DIR.iterdir():
    if not rdir.is_dir():
        continue
    result_paths[rdir.name] = {}
    for k, v in res.items():
        p = rdir / v
        if not p.exists():
            msg = f"File {p} not found"
            raise FileNotFoundError(msg)
        result_paths[rdir.name][k] = p


def natural_sort(text: str) -> list[int | str]:
    """Sort alphanumerical strings naturally."""

    def a2i(_text: str) -> int | str:
        return int(_text) if _text.isdigit() else _text

    return [a2i(t) for t in re.split(r"(\d+)", text)]


result_paths = {
    k: result_paths[k] for k in sorted(result_paths.keys(), key=natural_sort)
}

pprint(result_paths)  # noqa: T203

## Arrange Data

In [ ]:
lagrange_markers = {}

for label, paths in result_paths.items():
    ds_path = paths.get("dual_solver")
    if ds_path:
        ds_df = pd.read_csv(paths["dual_solver"], sep="\t")
        lagrange_markers[label] = list(ds_df.i.unique().tolist())
    else:
        lagrange_markers[label] = []

In [ ]:
lower_bounds = {}
lower_bounds_relative = {}

for label, paths in result_paths.items():
    bounds_df = pd.read_csv(paths["bounds"], sep="\t")
    ub = bounds_df["ub_r"].array[-1]
    lbs_abs = bounds_df["lb"].tolist()
    lbs_rel = bounds_df["lb"].div(ub).tolist()
    lbs_abs.pop()
    lbs_rel.pop()
    lower_bounds[label] = lbs_abs
    lower_bounds_relative[label] = lbs_rel

In [ ]:
runtimes = {}

for label, paths in result_paths.items():
    with paths["runtime"].open() as runtime_file:
        runtime_data = runtime_file.read()
        runtimes[label] = json.loads(runtime_data)

iter_durations = defaultdict(list)
iterations = {}

for label, runtime in runtimes.items():
    iter_list = [i + 1 for i in range(len(lower_bounds[label]))]
    iterations[label] = iter_list
    for i in iter_list:
        duration = sum(
            value
            for key, value in runtime.items()
            if f"i{i}" == key.split("_")[-1]
        )
        iter_durations[label].append(duration)

# Determine the cumulative duration of iterations
iter_durations_cum = {}
for label, durations in iter_durations.items():
    iter_durations_cum[label] = [
        sum(durations[:i]) for i in range(1, len(durations) + 1)
    ]

## Plot

### Graphs

In [ ]:
def plot_convergence(  # noqa: PLR0913
    labels: Iterable[str],
    durations: dict[str, list[float]],
    bounds: dict[str, list[float]],
    markers: dict[str, list[int]],
    xlabel: str,
    ylabel: str,
    title: str,
    hline: float | None = None,
    alt_labels: dict[str, str] | None = None,
    filename: str | None = None,
) -> None:
    """Plot convergence of lower bounds."""
    fig = plt.figure(dpi=600, facecolor="w", edgecolor="k")
    ax = fig.gca()
    ax.xaxis.get_major_locator().set_params(integer=True)

    plt.title(title)
    ax.set(xlabel=xlabel, ylabel=ylabel)

    if hline is not None:
        plt.axhline(hline, color="red", linestyle="-")

    alt_labels = alt_labels or {}

    z = 0
    for label, color in zip(
        labels, mcolors.TABLEAU_COLORS.keys(), strict=False
    ):
        y = [0] + bounds[label]
        iterations = list(range(len(y)))

        m = markers[label]
        if m and len(iterations) < m[-1]:
            m = m[:-1]

        if durations:
            x = [0] + durations[label]
            plt.plot(
                x, y, ls="-", label=alt_labels.get(label, label), color=color
            )
            z += 1
            for i in m:
                plt.scatter(x[i], y[i], color=color, marker="o")
            z += 1
        else:
            plt.plot(
                iterations,
                y,
                "o",
                ls="-",
                color=color,
                markevery=m,
                label=alt_labels.get(label, label),
            )

    plt.xlim(left=0)
    plt.ylim(bottom=0)

    plt.legend(loc=4)

    if filename:
        plt.savefig(OUT_DIR / filename)

    plt.show()

In [ ]:
plot_convergence(
    labels=result_paths,
    durations={},
    bounds=lower_bounds_relative,
    markers=lagrange_markers,
    xlabel="Iteration",
    ylabel=r"Relative lower bound $\left[\frac{1}{\overline{v}}\right]$",
    title="Lower Bound vs. SDDiP Iterations",
    hline=1,
    filename="convergence_relative.png",
)

In [ ]:
plot_convergence(
    labels=result_paths,
    durations={},
    bounds=lower_bounds,
    markers=lagrange_markers,
    xlabel="Iteration",
    ylabel="Lower bound",
    title="Lower Bound vs. SDDiP Iterations",
    filename="convergence_absolute.png",
)

In [ ]:
plot_convergence(
    labels=result_paths,
    durations=iter_durations_cum,
    bounds=lower_bounds_relative,
    markers=lagrange_markers,
    xlabel="Time past [s]",
    ylabel=r"Relative lower bound $\left[\frac{1}{\overline{v}}\right]$",
    title="Lower Bound vs. Time Past",
    hline=1,
    filename="convergence_time_relative.png",
)

In [ ]:
plot_convergence(
    labels=result_paths,
    durations=iter_durations_cum,
    bounds=lower_bounds,
    markers=lagrange_markers,
    xlabel="Time past [s]",
    ylabel="Lower bound",
    title="Lower Bound vs. Time Past",
    filename="convergence_time_absolute.png",
)

### Bar Charts

In [ ]:
fig = plt.figure(dpi=600, facecolor="w", edgecolor="k")
ax = fig.gca()

hatches = ["/", "\\", "|", "-", "+", "x", "o", "O", ".", "*"][
    : len(result_paths)
]

bar_labels = [ALT_LABELS.get(label, label) for label in result_paths]
bar_values = [(1 - lower_bounds_relative[label][-1]) for label in result_paths]

plt.bar(
    bar_labels, bar_values, color=mcolors.TABLEAU_COLORS.keys(), hatch=hatches
)

ax.set(ylabel=r"Gap $\left[\frac{1}{\overline{v}}\right]$")

plt.title(r"Optimality Gap")

plt.savefig(OUT_DIR / "optimality_gap_bars.png")
plt.show()